In [1]:
import numpy as np
import pandas as pd
import torch
from WideAndDeep_pytorch.wide_deep import WideAndDeepModel
from WideAndDeep_pytorch.avazu import AvazuDataset
from WideAndDeep_pytorch.train import train,test,EarlyStopper
from torch.utils.data import DataLoader
from IPython.core.interactiveshell import  InteractiveShell
InteractiveShell.ast_node_interactivity='all'
pd.set_option('max_columns',600)
pd.set_option('max_rows',500)

### 获取数据集与模型

In [2]:
#先读100w个存成csv，再用AvazuDataset加载
# df=pd.read_csv('./data/train.gz',compression='gzip',nrows=1000000)
# df.to_csv('./data/train.csv',index=False)
dataset=AvazuDataset('./data/train.csv',rebuild_cache=False)
model=WideAndDeepModel(dataset.field_dims, embed_dim=16,mlp_dims=(16, 16), dropout=0.2)

打印deep部分的模型架构:
Sequential(
  (0): Linear(in_features=352, out_features=16, bias=True)
  (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.2, inplace=False)
  (4): Linear(in_features=16, out_features=16, bias=True)
  (5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): Dropout(p=0.2, inplace=False)
  (8): Linear(in_features=16, out_features=1, bias=True)
)


In [10]:
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,ddd2926e,44956a24,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,96809ac8,711ee120,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,b3cf8def,8a4875bd,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,e8275b8f,6332421a,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,07d7df22,a99f214a,9644d0bf,779d90c2,1,0,18993,320,50,2161,0,35,-1,157


### 数据集拆分并用DataLoader加载

In [3]:
#按8:1:1比例拆分为训练集、验证集、测试集
train_length = int(len(dataset) * 0.8)
valid_length = int(len(dataset) * 0.1)
test_length = len(dataset) - train_length - valid_length
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(
    dataset, (train_length, valid_length, test_length))

#利用DataLoader加载，每个batch_size=256，num_workers表示可以多线程处理，里面包含了yield机制
train_data_loader = DataLoader(train_dataset, batch_size=256, num_workers=0)
valid_data_loader = DataLoader(valid_dataset, batch_size=256, num_workers=0)
test_data_loader = DataLoader(test_dataset, batch_size=256, num_workers=0)

### 开始训练模型

In [4]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001, weight_decay=0.000001)
#num_trials:表示尝试num_trials次后，如果没有提升就提前终止训练
#save_path：表示每次最优模型的存放路径
early_stopper = EarlyStopper(num_trials=2, save_path='result/wd_model_001.pt')
#开始训练
for epoch_i in range(100):
    train(model, optimizer, train_data_loader, criterion, device=None)
    auc_train = test(model, train_data_loader, device=None)
    auc_valid = test(model, valid_data_loader, device=None)
    auc_test = test(model, test_data_loader, device=None)
    print('第{}个epoch结束：'.format(epoch_i))
    print('训练集AUC:{}'.format(auc_train))
    print('验证集AUC:{}'.format(auc_valid))
    print('测试集AUC:{}'.format(auc_test))
    if not early_stopper.is_continuable(model, auc_valid):
        print('验证集上AUC的最高值是:{}'.format(early_stopper.best_accuracy))
        break

  0%|                                                                                         | 0/3125 [00:00<?, ?it/s]

第0个epoch结束：
训练集AUC:0.8042512026999078
验证集AUC:0.7592977266605292
测试集AUC:0.7620804385427883


  0%|                                                                                         | 0/3125 [00:00<?, ?it/s]

第1个epoch结束：
训练集AUC:0.8317456099809597
验证集AUC:0.7659468827502861
测试集AUC:0.7689926439285815


  0%|                                                                                         | 0/3125 [00:00<?, ?it/s]

第2个epoch结束：
训练集AUC:0.8369985987101622
验证集AUC:0.7626352457105574
测试集AUC:0.7650528218826029


100%|███████████████████████████████████████████████████████████████████████████████| 391/391 [00:01<00:00, 237.20it/s]

第3个epoch结束：
训练集AUC:0.8351362270370601
验证集AUC:0.7594660354036955
测试集AUC:0.7614059529112722
验证集上AUC的最高值是:0.7659468827502861
